In [1]:
import numpy as np
import pandas as pd
from quantfreedom.indicators.talib_ind import from_talib
import plotly.graph_objects as go
from IPython import get_ipython

In [6]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == "ZMQInteractiveShell":
            return True  # Jupyter notebook or qtconsole
        elif shell == "TerminalInteractiveShell":
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False  # Probably standard Python interpreter
is_notebook()

True